In [ ]:
Okay, so here it is - the first Jupyter notebook. 
As a programmer in my past life and a strategy consultant for a while now, I feel strongly that is the future of consulting and most of knowledge economy. 
More on that later!
For now, what we have at hand is a hobby project.
Along with my friend, Arpan, I am trying to explore an institutional way to understand leanings of media on crucial matters.
We start with a topical one - is hydroxychloraquin (ain't that a mouthful) effective for Covid19? 

In [ ]:
So, starting off, I want to test out how I can observe media quickly. 
As someone has said, good programmers develop algorithms, great programmers search github.
Shamelessly borrowing from the good folks at newpaper library, here is a quick render of summarizing a web article using this beautiful library. 
Hat tip folks!

In [13]:
import nltk
from newspaper import Article
article = Article('https://www.thehindu.com/sci-tech/health/coronavirus-hydroxychloroquine-not-effective-against-covid-19-says-study/article31406034.ece')
article.download()
article.parse()
article.nlp()
print(article.authors)
print(article.publish_date)
article.keywords
print(article.summary)


['As Many As Districts Have At Least One Case']
None
Hydroxychloroquine, the antimalarial drug which the U.S. President Donald Trump called a “game changer” against coronavirus (COVID-19), is not effective against the disease, according to a study.
The yet to be peer-reviewed study, published in the preprint server medRxiv, made a retrospective analysis of patients hospitalised with COVID-19 in all the Veterans Health Administration medical centers across the U.S.
According to the researchers, hydroxychloroquine, alone or in combination with azithromycin, is being widely used in COVID-19 therapy based on anecdotal evidence.
Also Read Coronavirus | Pharma units in limbo amid confusion over hydroxychloroquine exportsBased on the findings, the scientists said there is no evidence that use of hydroxychloroquine reduced the risk of mechanical ventilation in patients hospitalised with COVID-19.
“The findings from this retrospective study suggest caution in using hydroxychloroquine in hospita

In [ ]:
That worked well! 
I looked at the summary code under the hood and it basically scores the sentences with the highest appreance of keywords (words appearing the most and not the usual stopwords) as high and then builds a summary taking the highest ranked sentences. Quite elegant but something to be played around with the NLP inclined.

In [45]:
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse
from urllib.parse import urlparse
g_clean = [ ] 
url ='https://www.google.com/search?source=hp&ei=WijKXvTlCJHf9QPXm6D4DQ&q=is+hydroxychloroquine+working+for+covid+19&oq=is+hydroxychloroquine+working+for+&gs_lcp=CgZwc3ktYWIQAxgAMgIIADoFCAAQgwE6BwgAEEYQ_wE6DggAEOoCELQCEJoBEOUCUOYJWPQ2YIJAaAVwAHgAgAGOAYgBwxCSAQQwLjE4mAEAoAEBqgEHZ3dzLXdperABBg&sclient=psy-ab'
    
html = requests.get(url)
if html.status_code==200:
    soup = BeautifulSoup(html.text, 'lxml')
    a = soup.find_all('a') 
    for i in a:
        k = i.get('href')
        print(k)            
            

/?sa=X&ved=0ahUKEwi0zJ78gszpAhVMwzgGHYGZCC0QOwgC
/?output=search&ie=UTF-8&sa=X&ved=0ahUKEwi0zJ78gszpAhVMwzgGHYGZCC0QPAgE
/search?q=is+hydroxychloroquine+working+for+covid+19&ie=UTF-8&gbv=1&sei=cCjKXrTUAsyG4-EPgbOi6AI
/search?q=is+hydroxychloroquine+working+for+covid+19&ie=UTF-8&source=lnms&tbm=nws&sa=X&ved=0ahUKEwi0zJ78gszpAhVMwzgGHYGZCC0Q_AUICCgB
/search?q=is+hydroxychloroquine+working+for+covid+19&ie=UTF-8&source=lnms&tbm=shop&sa=X&ved=0ahUKEwi0zJ78gszpAhVMwzgGHYGZCC0Q_AUICSgC
/search?q=is+hydroxychloroquine+working+for+covid+19&ie=UTF-8&source=lnms&tbm=isch&sa=X&ved=0ahUKEwi0zJ78gszpAhVMwzgGHYGZCC0Q_AUICigD
https://maps.google.com/maps?q=is+hydroxychloroquine+working+for+covid+19&gs_lcp=CgZwc3ktYWIQAxgAMgIIADoFCAAQgwE6BwgAEEYQ_wE6DggAEOoCELQCEJoBEOUCUOYJWPQ2YIJAaAVwAHgAgAGOAYgBwxCSAQQwLjE4mAEAoAEBqgEHZ3dzLXdperABBg&um=1&ie=UTF-8&sa=X&ved=0ahUKEwi0zJ78gszpAhVMwzgGHYGZCC0Q_AUICygE
/search?q=is+hydroxychloroquine+working+for+covid+19&ie=UTF-8&source=lnms&tbm=vid&sa=X&ved=0ahUKEwi0zJ78g

In [ ]:
Phew, that seemed to work but the render is not pretty at all. So, onwards to trying to get this code correct

In [49]:
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse
from urllib.parse import urlparse
g_clean = [ ] 
url ='https://www.google.com/search?source=hp&ei=WijKXvTlCJHf9QPXm6D4DQ&q=is+hydroxychloroquine+working+for+covid+19&oq=is+hydroxychloroquine+working+for+&gs_lcp=CgZwc3ktYWIQAxgAMgIIADoFCAAQgwE6BwgAEEYQ_wE6DggAEOoCELQCEJoBEOUCUOYJWPQ2YIJAaAVwAHgAgAGOAYgBwxCSAQQwLjE4mAEAoAEBqgEHZ3dzLXdperABBg&sclient=psy-ab'
    
html = requests.get(url)
if html.status_code==200:
    soup = BeautifulSoup(html.text, 'lxml')
    a = soup.find_all('a') 
    for i in a:
        k = i.get('href')
        try:
            m = re.search("(?P<url>https?://[^\s]+)", k)
            n = m.group(0)
            rul = n.split('&')[0]
            domain = urlparse(rul)
            if(re.search('google.com', domain.netloc)):
                continue
            else:
                g_clean.append(rul)
        except:
            continue
            
            
for x in range(len(g_clean)):
    print(g_clean[x])


https://www.cdc.gov/coronavirus/2019-ncov/hcp/therapeutic-options.html
https://www.goodrx.com/blog/coronavirus-medicine-chloroquine-hydroxychloroquine-as-covid19-treatment/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7102549/
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7102549/%23sec0001title
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7102549/%23sec0002title
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7102549/%23sec0018title
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7102549/%23sec0013title
https://www.thelancet.com/journals/laninf/article/PIIS1473-3099(20)30313-3/fulltext
https://www.statnews.com/pharmalot/2020/04/28/covid19-coronavirus-hydroxycholoroquine-lupus-trump/
https://www.bbc.com/news/51980731
https://theprint.in/health/hcq-breakthrough-icmr-finds-its-effective-in-preventing-coronavirus-expands-its-use/427583/
https://www.genengnews.com/news/novartis-plans-phase-iii-trial-of-hydroxychloroquine-for-covid-19/
https://www.sciencenews.org/article/coronavirus-covid-19-polit

In [ ]:
That is good progress - makes you wonder how it has become to code! 
Although also makes you wonder how people can easily copy paste code, and pass off as the developers of that. 
Phew, and I had tried to laboriously read Knuth in 2008/09